In [9]:
import json
import numpy
import math

In [10]:
# write file 

data = {} 
data['pegawai'] = []
data['dirawat'] = []
data['fasilitas'] = []
data['p'] = 4
data['b'] = 8092
data['pegawai'].append({  
    'tabel' : ["no_ktp", "tgl_lahir", "gender", "pendidikan"],
    'r' : 128,
    'n' : 100000,
    'v' : 12,
    'br' : 10000
})
data['dirawat'].append({  
    'tabel' : ["id_dirawat","tgl_dirawat", "status", "periode"],
    'r' : 250,
    'n' : 15000,
    'v' : 6,
    'br' : 1000
})
data['fasilitas'].append({  
    'tabel' : ["no_inventaris", "nama", "jenis", "tgl_dibeli", "pemakaian"],
    'r' : 150,
    'n' : 12500,
    'v' : 8,
    'br' : 1000
})

with open('datatry.json', 'w') as outfile:  
    json.dump(data, outfile)

In [11]:
# read file
with open('datatry.json') as json_file:  
    data = json.load(json_file)

print(data['pegawai'][0]['tabel'])

['no_ktp', 'tgl_lahir', 'gender', 'pendidikan']


In [12]:
print('Menu 1 : BFR dan Fan Out Ratio')
print('')

bfr_p = math.floor(data['b'] / data['pegawai'][0]['r'])
for_p = math.floor(data['b'] / (data['pegawai'][0]['v'] + data['p']))
print('BFR Pegawai : ', bfr_p)
print('Fan Out Rasio Pegawai : ', for_p)

bfr_d = math.floor(data['b'] / data['dirawat'][0]['r'])
for_d = math.floor(data['b'] / (data['dirawat'][0]['v'] + data['p']))
print('BFR Pegawai : ', bfr_d)
print('Fan Out Rasio Dirawat : ' , for_d)

bfr_f = math.floor(data['b'] / data['fasilitas'][0]['r'])
for_f = math.floor(data['b'] / (data['fasilitas'][0]['v'] + data['p']))
print('BFR Pegawai : ', bfr_f)
print('Fan Out Rasio Dirawat : ', for_f)

Menu 1 : BFR dan Fan Out Ratio

BFR Pegawai :  63
Fan Out Rasio Pegawai :  505
BFR Pegawai :  32
Fan Out Rasio Dirawat :  809
BFR Pegawai :  53
Fan Out Rasio Dirawat :  674


In [13]:
print('Menu 2 : Jumlah Blok')
print('')

tabel_p = math.ceil(data['pegawai'][0]['n'] / bfr_p)
idx_p = math.ceil(data['pegawai'][0]['n'] / for_p)
print('Tabel Data Pegawai :', tabel_p, ' blok')
print('Indeks Pegawai :', idx_p, ' blok')

tabel_d = math.ceil(data['dirawat'][0]['n'] / bfr_d)
idx_d = math.ceil(data['dirawat'][0]['n'] / for_d)
print('Tabel Data Dirawat :', tabel_d, ' blok')
print('Indeks Dirawat:', idx_d, ' blok')

tabel_f = math.ceil(data['fasilitas'][0]['n'] / bfr_f)
idx_f = math.ceil(data['fasilitas'][0]['n'] / for_f)
print('Tabel Data Fasilitas :', tabel_f, ' blok')
print('Indeks Fasilitas :', idx_f, ' blok')

Menu 2 : Jumlah Blok

Tabel Data Pegawai : 1588  blok
Indeks Pegawai : 199  blok
Tabel Data Dirawat : 469  blok
Indeks Dirawat: 19  blok
Tabel Data Fasilitas : 236  blok
Indeks Fasilitas : 19  blok


In [14]:
print('Menu 3 : Pencarian Rekord')
print('')

def fanoutRasio(tabel):
    return math.floor(data['b'] / (data[tabel][0]['v'] + data['p']))

def blockingFactor(t):
    return math.floor(data['b'] / data[t][0]['r'])

def jumBlokIndeks(r, t):
    if (r>data[t][0]['r']):
        return 'record tidak tersedia'
    else :
        fanout = fanoutRasio(t)
        return math.ceil(r / fanout)

def jumBlokNonIndeks(r, t):
    if (r>data[t][0]['r']):
        return 'record tidak tersedia'
    else :
        bfr = blockingFactor(t)
        return math.ceil(r / bfr)
    

rekord = int(input("Cari Rekord ke : "))
tabel = input("Nama Tabel : ")

blok_i = jumBlokIndeks(rekord, tabel)

print ('Menggunakan indeks, jumlah blok yang diakses : ', blok_i)
print ('Tanpa indeks, jumlah blok yang diakses : ', jumBlokNonIndeks(rekord, tabel))

Menu 3 : Pencarian Rekord

Cari Rekord ke : 300
Nama Tabel : pegawai
Menggunakan indeks, jumlah blok yang diakses :  record tidak tersedia
Tanpa indeks, jumlah blok yang diakses :  record tidak tersedia


In [15]:
print('Menu 4 : QEP dan COST')
print('')

#qep where
array_query = {}
array_query['kolom'] = ['no_inventaris','nama']
array_query['tabel'] = ['fasilitas']
array_query['where'] = ['no_inventaris = 1']
def qep(query):
    if (query['kolom'][0] != '*'):
        return { 
            'projection' : query['kolom'],
            'selection' : query['where'],
            'tabel' : query['tabel']
        }
    else:
        return {
            'selection' : query['where'],
            'tabel' : query['tabel']
        }

#cost where, diisi ya geng hehe
def costA1nkey(t):
    return data[t][0]['br']

def costA1key(t):
    return data[t][0]['br']/2

def costA2(t):
    return math.ceil(math.log(data[t][0]['br'],fanoutRasio(t))) + 1

def costBNLJ(inner,outer):
    return data[inner][0]['br']*data[outer][0]['br'] + data[outer][0]['br']

#qep join, diisi ya geng hehe

#cost join, diisi ya geng hehe

#tes qep where
if(array_query['kolom'][0]!='*'):
    print('PROJECTION', qep(array_query)['projection'][0], ',' , qep(array_query)['projection'][1])
print('SELECTION', qep(array_query)['selection'][0])
print(qep(array_query)['tabel'][0])

h = costA1key(array_query['tabel'][0])
print(h)
print(costBNLJ('fasilitas','pegawai'))

Menu 4 : QEP dan COST

PROJECTION no_inventaris , nama
SELECTION no_inventaris = 1
fasilitas
500.0
10010000
